In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import wandb 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 40

class Lang:
    def __init__(self, name):
        self.name = name
        # self.word2index = {}
        # self.word2count = {}
        # self.index2word = {0: "SOS", 1: "EOS"}
        # self.n_words = 2  # Count SOS and EOS
        self.letter_index = {}
        self.letter_count = {}
        self.index_letter = {0: "SOS", 1: "EOS"}
        self.n_letters = 2  # Count SOS and EOS


    def add_word(self, word):
        # for word in sentence.split(' '):
        #     self.addWord(word)
        for letter in word:
            self.add_letter(letter)

    def add_letter(self, letter):
        # if word not in self.word2index:
        #     self.word2index[word] = self.n_words
        #     self.word2count[word] = 1
        #     self.index2word[self.n_words] = word
        #     self.n_words += 1
        # else:
        #     self.word2count[word] += 1
        if letter not in self.letter_index:
            self.letter_index[letter] = self.n_letters
            self.letter_count[letter] = 1
            self.index_letter[self.n_letters] = letter
            self.n_letters += 1
        else:
            self.letter_count[letter] += 1

In [4]:
def read_words(output_lang, type = 'train', reverse = False):
    lines = open('aksharantar_sampled/%s/%s_%s.csv' % (output_lang,output_lang,type), encoding='utf-8').read().strip().split('\n')
    pairs = [[s for s in l.split(',')] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
    else:
        pairs = [list(p) for p in pairs]
    return pairs

def init_lang(lang1, lang2, type = 'train', reverse = False):
    pairs = read_words(lang2, type, reverse)
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    char = 'a'
    for i in range(26):
        input_lang.add_letter(char)
        char = chr(ord(char) + 1)
    char = '\u0900'
    for i in range(128):
        output_lang.add_letter(char)
        char = chr(ord(char) + 1)

    
    return input_lang, output_lang, pairs

In [5]:
pairs = read_words('mar',type = 'train')
for p in pairs:
    MAX_LENGTH = max(MAX_LENGTH, len(p[1]))
print("MAX_LENGTH: ", MAX_LENGTH)

MAX_LENGTH:  40


In [6]:
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')]
def word_to_index(lang, word):
    return [lang.letter_index[letter] for letter in word]


# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def word_to_tensor(lang, word):
    indexes = word_to_index(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


# def tensorsFromPair(pair):
#     input_tensor = tensorFromSentence(input_lang, pair[0])
#     target_tensor = tensorFromSentence(output_lang, pair[1])
#     return (input_tensor, target_tensor)

def pair_to_tensor(input_lang,output_lang, pair):
    input_tensor = word_to_tensor(input_lang, pair[0])
    target_tensor = word_to_tensor(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, type = 'gru', nonlinearity = 'tanh'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.type = type
        if self.type == 'gru':
            self.gru = nn.GRU(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'lstm':
            self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'rnn':
            self.rnn = nn.RNN(hidden_size, hidden_size, num_layers = num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        if self.type == 'gru':
            output, hidden = self.gru(output, hidden)
        elif self.type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        elif self.type == 'rnn':
            output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.num_layers , 1, self.hidden_size, device=device)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, nonlinearity = 'tanh', dropout_p = 0.1, type = 'gru'):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # self.dropout_p = dropout_p
        self.embedding = nn.Embedding(output_size, hidden_size)
        # self.dropout = nn.Dropout(self.dropout_p)
        self.num_layers = num_layers
        self.type = type
        if self.type == 'gru':
            self.gru = nn.GRU(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'lstm':
            self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'rnn':
            self.rnn = nn.RNN(hidden_size, hidden_size, num_layers = num_layers)
            
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        # output = self.dropout(output)
        output = F.relu(output)
        if self.type == 'gru':
            output, hidden = self.gru(output, hidden)
        elif self.type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        elif self.type == 'rnn':
            output, hidden = self.rnn(output, hidden)
        
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, nonlinearity = 'tanh', dropout_p=0.1, max_length=MAX_LENGTH, type = 'gru'):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.num_layers = num_layers
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.type = type
        
        if self.type == 'gru':
            self.gru = nn.GRU(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'lstm':
            self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers = num_layers)
        elif self.type == 'rnn':
            self.rnn = nn.RNN(hidden_size, hidden_size, num_layers = num_layers)

        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        if self.type == 'gru':
            output, hidden = self.gru(output, hidden)
        elif self.type == 'lstm':
            output, hidden = self.lstm(output, hidden)
        elif self.type == 'rnn':
            output, hidden = self.rnn(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [36]:
class Transliterator():
    def __init__(self, encoder_hp, decoder_hp, attn = True):
        self.encoder = EncoderRNN(encoder_hp['input_size'], encoder_hp['hidden_size'], encoder_hp['num_layers'], type = encoder_hp['type']).to(device)
        self.attn = attn
        if attn:
            self.decoder = AttnDecoderRNN(decoder_hp['hidden_size'], decoder_hp['output_size'], decoder_hp['num_layers'], type = decoder_hp['type']).to(device)
        else:
            self.decoder = DecoderRNN(decoder_hp['hidden_size'], decoder_hp['output_size'], decoder_hp['num_layers'], type = decoder_hp['type']).to(device)

    def train(self, input_tensor, target_tensor, max_length=MAX_LENGTH):
        encoder_hidden = self.encoder.initHidden()

        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        encoder_outputs = torch.zeros(max_length, self.encoder.hidden_size, device=device)

        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden = self.encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden[encoder_hidden.shape[0] - 1].unsqueeze(0)
        decoded_word = ''
        for i in range(self.decoder.num_layers - 1):
            decoder_hidden = torch.cat((decoder_hidden, encoder_hidden[encoder_hidden.shape[0] - 1 - i].unsqueeze(0)), 0)

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False
        
        if use_teacher_forcing:
            for di in range(target_length):
                if self.attn:
                    decoder_output, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                else:
                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  
                loss += self.criterion(decoder_output, target_tensor[di])
                decoded_word += self.output_lang.index_letter[topi.item()]
                decoder_input = target_tensor[di]
        else:
            for di in range(target_length):
                if self.attn:
                    decoder_output, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                else:
                    decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  
                loss += self.criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break
                decoded_word += self.output_lang.index_letter[topi.item()]
                
        
        loss.backward()

        self.encoder_optimizer.step()
        self.decoder_optimizer.step()
        
        return loss.item() / target_length, decoded_word 
    
    def fit( self, train_io_pair, val_io_pair, input_lang, output_lang, n_epochs,optimizer = 'adam', criterion = nn.NLLLoss(), print_every=1000, learning_rate=0.0001, teacher_forcing_ratio = 0.5):
        self.input_lang = input_lang
        self.output_lang = output_lang

        if(optimizer == 'adam'):
            self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=learning_rate)
            self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=learning_rate)
        elif(optimizer == 'sgd'):
            self.encoder_optimizer = optim.SGD(self.encoder.parameters(), lr=learning_rate)
            self.decoder_optimizer = optim.SGD(self.decoder.parameters(), lr=learning_rate)
            
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.n_epochs = n_epochs
        self.print_every = print_every
        
        self.all_losses = []
        self.total_loss = 0

        # for i in range(len(train_io_pair)):
        #     train_io_pair[i] = pair_to_tensor(train_io_pair[i])
        training_pairs = [pair_to_tensor(self.input_lang, self.output_lang, train_io_pair[i]) for i in range(len(train_io_pair))]
        
        for epc in range(1,n_epochs + 1):
            correct = 0
            for inp in range(len(train_io_pair)):
                input_tensor = training_pairs[inp][0]
                target_tensor = training_pairs[inp][1]
                loss, decoder_output = self.train(input_tensor, target_tensor)
                self.total_loss += loss
                self.all_losses.append(loss)
                if(decoder_output == train_io_pair[inp][1]):
                    correct += 1
            print("Epoch: ", epc, " Loss: ", self.total_loss / len(train_io_pair),"Training Accuracy: ", correct/len(train_io_pair),"Validation Accuracy: ", self.eval(val_io_pair))
            self.total_loss = 0

    def eval(self, io_pairs):
        correct = 0
        for i in range(len(io_pairs)):
            output, _ = self.predict(io_pairs[i][0])
            if output == io_pairs[i][1]:
                correct += 1
        return correct / len(io_pairs)
    
    def predict(self, input_word, max_length=MAX_LENGTH):
        with torch.no_grad():
            input_tensor = word_to_tensor(self.input_lang, input_word)
            input_length = input_tensor.size()[0]
            encoder_hidden = self.encoder.initHidden()

            encoder_outputs = torch.zeros(max_length, self.encoder.hidden_size, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = self.encoder(input_tensor[ei],
                                                        encoder_hidden)
                encoder_outputs[ei] += encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            decoder_hidden = encoder_hidden[encoder_hidden.shape[0] - 1].unsqueeze(0)
            for i in range(self.decoder.num_layers - 1):
                decoder_hidden = torch.cat((decoder_hidden, encoder_hidden[encoder_hidden.shape[0] - 1 - i].unsqueeze(0)), 0)

            # decoded_words = []
            decoded_word = ''
            decoder_attentions = torch.zeros(max_length, max_length)

            for di in range(max_length):
                if self.attn:
                    decoder_output, decoder_hidden, decoder_attention = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                    decoder_attentions[di] = decoder_attention.data
                else:
                    decoder_output, decoder_hidden = self.decoder(
                    decoder_input, decoder_hidden)
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    # decoded_words.append('<EOS>')
                    # decoded_word += '<EOS>'
                    break
                else:
                    # decoded_words.append(output_lang.letter_index[topi.item()])
                    decoded_word += self.output_lang.index_letter[topi.item()]
                    # print(output_lang.index_letter[topi.item()])

                decoder_input = topi.squeeze().detach()

            return decoded_word, decoder_attentions[:di + 1]



    


In [37]:
input_lang, output_lang, train_pairs = init_lang( 'eng','mar', type = 'train', reverse = False)
validation_pairs = read_words('mar', type = 'valid')
# print(input_lang.n_letters)
encoder_hp = {'input_size': input_lang.n_letters, 'hidden_size': 256, 'num_layers': 3, 'type': 'gru'}
decoder_hp = {'hidden_size': 256, 'output_size': output_lang.n_letters, 'num_layers': 2, 'type': 'gru'}

model = Transliterator(encoder_hp=encoder_hp, decoder_hp=decoder_hp, attn = False)

model.fit(train_pairs, validation_pairs, input_lang, output_lang, n_epochs = 5, learning_rate = 0.0003, teacher_forcing_ratio = 0.5)

Epoch:  1  Loss:  1.278363156748678 Training Accuracy:  0.0368359375 Validation Accuracy:  0.1748046875
Epoch:  2  Loss:  0.8726902751648318 Training Accuracy:  0.0878125 Validation Accuracy:  0.240478515625
Epoch:  3  Loss:  0.7840190025630573 Training Accuracy:  0.1137109375 Validation Accuracy:  0.264404296875
Epoch:  4  Loss:  0.7397998276464685 Training Accuracy:  0.127421875 Validation Accuracy:  0.25830078125


In [13]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
wandb.init(project="aksharantar", name="gru_attn")

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'encoder_hidden_size': {
            'values': [64, 128, 256]   },
        'encoder_num_layers': { 
            'values': [1, 2, 3]   },
        'encoder_type': {'values' : ['gru', 'lstm', 'rnn']},
        'decoder_hidden_size': {
            'values': [64, 128, 256]   },
        'decoder_num_layers': {
            'values': [1, 2, 3]   },
        'decoder_type': {'values' : ['gru', 'lstm', 'rnn']},
        'learning_rate': {'values': [0.0001, 0.00001, 0.001, 0.0003]},
        'teacher_forcing_ratio': {'values': [0.5, 0.6, 0.7, 0.8, 0.9]},
        'optimizer' : {'values': ['adam', 'sgd']},
        'epochs' : { 'values' : [5,10]}
        
    }
}

sweep_id = wandb.sweep(sweep_config, project="aksharantar")

def train():
    config_defaults = {
        'encoder_hidden_size': 64,
        'encoder_num_layers': 1,
        'encoder_type': 'gru',
        'decoder_hidden_size': 64,
        'decoder_num_layers': 1,
        'decoder_type': 'gru',
        'learning_rate': 0.0001,
        'teacher_forcing_ratio': 0.5,
        'optimizer' : 'adam',
        'epochs' : 5
    }

    run = wandb.init(project="aksharantar", config=config_defaults)
    config = wandb.config
    name_str = config.encoder_type + '_' + config.decoder_type + '_' + str(config.encoder_hidden_size) + '_' + str(config.encoder_num_layers) + '_' + str(config.decoder_hidden_size) + '_' + str(config.decoder_num_layers) + '_' + str(config.learning_rate) + '_' + str(config.teacher_forcing_ratio) + '_' + config.optimizer + '_' + str(config.epochs)
    wandb.run.name = name_str
    model = Transliterator(
        encoder_hp={
            'input_size': input_lang.n_letters, 
            'hidden_size': config.encoder_hidden_size, 
            'num_layers': config.encoder_num_layers, 
            'type': config.encoder_type}, 
        decoder_hp={
            'hidden_size': config.decoder_hidden_size, 
            'output_size': output_lang.n_letters, 
            'num_layers': config.decoder_num_layers, 
            'type': config.decoder_type}, 
        attn = True)
    model.fit(train_pairs, validation_pairs, input_lang, output_lang, n_epochs = config.epochs, learning_rate = config.learning_rate, teacher_forcing_ratio = config.teacher_forcing_ratio)
    